In [1]:
import sys
import datetime as dt
import pandas as pd
import numpy as np
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../')
from envir import config

In [2]:
# read in datasets
spl = pd.read_csv(config.dataFol+'clean_spl/clean_spl.csv')
spl['sonyc_sensor_id'] = spl['sonyc_sensor_id'].str[10:22]
nexrad = pd.read_csv(config.dataFol+'clean_nexrad/clean_nexrad.csv')
yamnet = pd.read_csv(config.dataFol+'clean_yamnet/clean_yamnet.csv',
                     names = ['sonyc_sensor_id', 'timestamp', 'count_of_positive_predictions', 'pct_positive_predictions'])
weather = pd.read_csv(config.dataFol+'clean_noaa/clean_weather.csv')

In [3]:
# create a timestamp index
df = pd.DataFrame({'timestamp': pd.date_range(start="2017-01-01", end="2020-05-31", freq='T')})
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df = df[(df.month >=3) & (df.month <= 5)].reset_index() # filter for only the months of march through may
df.drop(columns = ['index', 'month'], inplace=True)
df['timestamp'] = df['timestamp'].astype('str').str[0:16]

In [4]:
df.head()

,timestamp
0,2017-03-01 00:00
1,2017-03-01 00:01
2,2017-03-01 00:02
3,2017-03-01 00:03
4,2017-03-01 00:04


In [5]:
# merge spl data to the time index
# seems like the coverage for available spl data is by the minute so no need to fill forward
sensor = spl.sonyc_sensor_id.unique().tolist()
ind_spl = []
for s in sensor:
    ind_spl.append(df.merge(spl[spl.sonyc_sensor_id==s], how='left', on='timestamp'))
spl = pd.concat(ind_spl)

In [6]:
spl.dropna()

,timestamp,sonyc_sensor_id,sonyc_sensor_name,dBAS_lin_mean,l1,l10,l5,l90
0,2017-03-01 00:00,b827eb815321,19 Washington Square North,63.886124,68.820496,66.593994,67.362000,56.379000
1,2017-03-01 00:01,b827eb815321,19 Washington Square North,64.185800,69.763600,67.471000,68.420000,56.598000
2,2017-03-01 00:02,b827eb815321,19 Washington Square North,61.480568,68.093100,65.466995,66.493000,54.978000
3,2017-03-01 00:03,b827eb815321,19 Washington Square North,64.721580,68.886700,65.381000,66.382996,58.070000
4,2017-03-01 00:04,b827eb815321,19 Washington Square North,63.277860,67.505200,66.451996,66.850500,56.489002
...,...,...,...,...,...,...,...,...
528476,2020-05-30 23:56,b827eb1685c7,Shimkin Reading Room,67.401970,75.113800,69.894000,71.537000,64.336000
528477,2020-05-30 23:57,b827eb1685c7,Shimkin Reading Room,64.602310,69.200500,66.481000,67.412000,60.527000
528478,2020-05-30 23:58,b827eb1685c7,Shimkin Reading Room,63.392600,69.294304,65.633995,67.433000,59.909000
528479,2020-05-30 23:59,b827eb1685c7,Shimkin Reading Room,65.883210,75.288300,67.982000,71.441505,60.800000


In [7]:
# clean up radar data from the 2 sites and merge to timeindex with fill forward
nexrad = nexrad.groupby('timestamp').mean().reset_index()
colnames = nexrad.columns.tolist()[1:]
newcolnames=["{}_{}".format('avg', i) for i in colnames]
nexrad.rename(columns=dict(zip(colnames, newcolnames)), inplace=True)
nexrad = df.merge(nexrad, how='left', on='timestamp')
nexrad.fillna(method='ffill', limit=3, inplace=True)
nexrad.fillna(method='bfill', limit=3, inplace=True)

In [8]:
nexrad.dropna()

,timestamp,avg_mtr_#/km/h,avg_mt_#/km,avg_height_m
0,2017-03-01 00:00,5.917225,44334.097307,277.105735
1,2017-03-01 00:01,5.917225,44334.097307,277.105735
2,2017-03-01 00:02,95.663130,25566.371530,162.629644
3,2017-03-01 00:03,95.663130,25566.371530,162.629644
4,2017-03-01 00:04,95.663130,25566.371530,162.629644
...,...,...,...,...
528476,2020-05-30 23:56,64.208468,309133.480263,125.879287
528477,2020-05-30 23:57,64.208468,309133.480263,125.879287
528478,2020-05-30 23:58,64.208468,309133.480263,125.879287
528479,2020-05-30 23:59,44.173976,557231.510044,102.227134


In [9]:
ind_yamnet = []
for s in sensor:
    temp_yamnet = df.merge(yamnet[yamnet.sonyc_sensor_id==s], how='left', on='timestamp')
    temp_yamnet.fillna(method='ffill', limit=60, inplace=True)
    ind_yamnet.append(temp_yamnet)
yamnet = pd.concat(ind_yamnet)

In [10]:
yamnet.dropna()

,timestamp,sonyc_sensor_id,count_of_positive_predictions,pct_positive_predictions
0,2017-03-01 00:00,b827eb815321,0.0,0.0
1,2017-03-01 00:01,b827eb815321,0.0,0.0
2,2017-03-01 00:02,b827eb815321,0.0,0.0
3,2017-03-01 00:03,b827eb815321,0.0,0.0
4,2017-03-01 00:04,b827eb815321,0.0,0.0
...,...,...,...,...
501237,2020-05-09 00:55,b827eb1685c7,0.0,0.0
501238,2020-05-09 00:56,b827eb1685c7,0.0,0.0
501239,2020-05-09 00:57,b827eb1685c7,0.0,0.0
501240,2020-05-09 00:58,b827eb1685c7,0.0,0.0


In [11]:
weather = df.merge(weather, how='left', on='timestamp')
weather.fillna(method='ffill', inplace=True)

In [12]:
weather.head()

,timestamp,temp_celcius,dewp_celcius,rh_percentage,wind_dir,wind_speed_mph,sea_level_pressure_mb,precipitation_mm,visibility_miles,gust_mph,peak_wind_gust_mph
0,2017-03-01 00:00,10.0,10.0,100.0,140.0,9.2,1021.0,0.0,0.25,NaN,NaN
1,2017-03-01 00:01,10.0,10.0,100.0,140.0,9.2,1021.0,0.0,0.25,NaN,NaN
2,2017-03-01 00:02,10.0,10.0,100.0,140.0,9.2,1021.0,0.0,0.25,NaN,NaN
3,2017-03-01 00:03,10.0,10.0,100.0,140.0,9.2,1021.0,0.0,0.25,NaN,NaN
4,2017-03-01 00:04,10.0,10.0,100.0,140.0,9.2,1021.0,0.0,0.25,NaN,NaN


In [13]:
df = df.merge(spl, how='left', on='timestamp')
df = df.merge(yamnet, how='left', on=['timestamp', 'sonyc_sensor_id'])
df = df.merge(nexrad, how='left', on='timestamp')
df = df.merge(weather, how='left', on='timestamp')

In [14]:
df.shape

(4864933, 23)

In [15]:
df.columns

Index(['timestamp', 'sonyc_sensor_id', 'sonyc_sensor_name', 'dBAS_lin_mean',
       'l1', 'l10', 'l5', 'l90', 'count_of_positive_predictions',
       'pct_positive_predictions', 'avg_mtr_#/km/h', 'avg_mt_#/km',
       'avg_height_m', 'temp_celcius', 'dewp_celcius', 'rh_percentage',
       'wind_dir', 'wind_speed_mph', 'sea_level_pressure_mb',
       'precipitation_mm', 'visibility_miles', 'gust_mph',
       'peak_wind_gust_mph'],
      dtype='object')

In [16]:
df.describe()

,dBAS_lin_mean,l1,l10,l5,l90,count_of_positive_predictions,pct_positive_predictions,avg_mtr_#/km/h,avg_mt_#/km,avg_height_m,temp_celcius,dewp_celcius,rh_percentage,wind_dir,wind_speed_mph,sea_level_pressure_mb,precipitation_mm,visibility_miles,gust_mph,peak_wind_gust_mph
count,2.518710e+06,2.518710e+06,2.518710e+06,2.518710e+06,2.518710e+06,1.461514e+06,1.461514e+06,2.488529e+06,2.488529e+06,2.395069e+06,4.864933e+06,4.864933e+06,4.864933e+06,4.864933e+06,4.864933e+06,4.864933e+06,4.864933e+06,4.864933e+06,4.846010e+06,4.851450e+06
mean,6.238949e+01,6.876742e+01,6.456622e+01,6.609451e+01,5.810649e+01,5.057221e-02,2.101016e-02,4.007453e+02,2.304174e+05,4.121091e+02,1.099828e+01,4.932099e+00,7.019154e+01,1.840023e+02,1.224281e+01,1.016584e+03,1.855730e-01,8.825422e+00,2.546381e+01,3.264877e+01
std,5.332214e+00,6.835326e+00,5.656009e+00,6.052198e+00,4.416002e+00,4.973876e-01,5.432447e-02,1.361774e+03,2.060973e+05,3.201020e+02,6.304199e+00,8.542202e+00,2.328657e+01,1.070982e+02,6.398025e+00,8.119597e+00,7.975082e-01,2.718339e+00,5.181909e+00,3.307807e+00
min,4.470492e+01,4.942840e+01,4.728500e+01,4.910500e+01,4.079000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.040482e-01,1.000000e+02,-8.890000e+00,-2.500000e+01,1.236000e+01,0.000000e+00,0.000000e+00,9.841000e+02,0.000000e+00,0.000000e+00,1.610000e+01,2.990000e+01
25%,5.864803e+01,6.425420e+01,6.057700e+01,6.197000e+01,5.502000e+01,0.000000e+00,0.000000e+00,8.956401e+00,6.953647e+04,1.759542e+02,6.720000e+00,-1.110000e+00,5.118000e+01,9.000000e+01,8.050000e+00,1.011400e+03,0.000000e+00,1.000000e+01,2.185000e+01,3.105000e+01
50%,6.186490e+01,6.853135e+01,6.431500e+01,6.590100e+01,5.756900e+01,0.000000e+00,0.000000e+00,3.271674e+01,1.583407e+05,3.173675e+02,1.111000e+01,6.110000e+00,7.325000e+01,1.800000e+02,1.150000e+01,1.016700e+03,0.000000e+00,1.000000e+01,2.415000e+01,3.220000e+01
75%,6.536185e+01,7.270940e+01,6.782000e+01,6.949200e+01,6.035800e+01,0.000000e+00,1.667000e-02,1.559238e+02,3.274902e+05,5.723243e+02,1.561000e+01,1.172000e+01,9.248000e+01,2.900000e+02,1.610000e+01,1.021800e+03,0.000000e+00,1.000000e+01,2.760000e+01,3.335000e+01
max,1.036288e+02,1.128047e+02,1.093930e+02,1.115665e+02,9.696300e+01,1.900000e+01,1.000000e+00,3.055570e+04,8.890434e+05,4.703494e+03,3.278000e+01,2.111000e+01,1.000000e+02,3.600000e+02,4.485000e+01,1.040600e+03,1.473000e+01,1.000000e+01,5.750000e+01,6.670000e+01


In [17]:
df['sonyc_sensor_name'].unique()

array(['19 Washington Square North', nan, 'Silver lab', 'Juan Carlos',
       'Shimkin Reading Room', '4 Washington Square North',
       'Kimmel Center'], dtype=object)

#### One Hot the Sensors

In [18]:
df['s_19WashingtonSquareNorth'] = df['sonyc_sensor_name'].apply(lambda x: 1 if x == '19 Washington Square North' else 0)
#df['s_silverLab'] = df['sonyc_sensor_name'].apply(lambda x: 1 if x == 'Silver lab' else 0)
df['s_juanCarlos'] = df['sonyc_sensor_name'].apply(lambda x: 1 if x == 'Juan Carlos' else 0)
df['s_shimkinReadingRoom'] = df['sonyc_sensor_name'].apply(lambda x: 1 if x == 'Shimkin Reading Room' else 0)
df['s_4WashingtonSquareNorth'] = df['sonyc_sensor_name'].apply(lambda x: 1 if x == '4 Washington Square North' else 0)
df['s_kimmelCenter'] = df['sonyc_sensor_name'].apply(lambda x: 1 if x == 'Kimmel Center' else 0)

In [20]:
df.rename(columns={'timestamp':'timestamp_utc'}, inplace=True)

In [21]:
df.columns

Index(['timestamp_utc', 'sonyc_sensor_id', 'sonyc_sensor_name',
       'dBAS_lin_mean', 'l1', 'l10', 'l5', 'l90',
       'count_of_positive_predictions', 'pct_positive_predictions',
       'avg_mtr_#/km/h', 'avg_mt_#/km', 'avg_height_m', 'temp_celcius',
       'dewp_celcius', 'rh_percentage', 'wind_dir', 'wind_speed_mph',
       'sea_level_pressure_mb', 'precipitation_mm', 'visibility_miles',
       'gust_mph', 'peak_wind_gust_mph', 's_19WashingtonSquareNorth',
       's_juanCarlos', 's_shimkinReadingRoom', 's_4WashingtonSquareNorth',
       's_kimmelCenter'],
      dtype='object')

In [22]:
df.dropna().to_csv(config.dataFol+'all_cleaned_wsp_sonyc_birds.csv', index=False)

In [23]:
df.dropna()['sonyc_sensor_name'].unique()

array(['19 Washington Square North', 'Juan Carlos',
       'Shimkin Reading Room', '4 Washington Square North',
       'Kimmel Center'], dtype=object)